<a href="https://colab.research.google.com/github/jbtuku/Data-Engineering/blob/main/%20Detecting%20Bias/Detecting_Bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

# Step 1: Mount your Google Drive
drive.mount('/content/drive')

# Step 2: Set your actual file path
file_path = '/content/drive/MyDrive/trimet_stopevents_2022-12-07.html'  # update if needed

# Step 3: Read and parse the HTML file
with open(file_path, 'r', encoding='utf-8') as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, 'html.parser')

# Step 4: Extract all tables and corresponding trip_id from h2 headings
initial_date = datetime(2020, 1, 1)
all_records = []

trip_headings = soup.find_all('h2')

for heading in trip_headings:
    trip_id = heading.text.strip().split()[-1]  # extract the trip number
    table = heading.find_next_sibling('table')
    if not table:
        continue

    # Extract header and rows
    headers = [th.text.strip() for th in table.find_all('th')]
    for row in table.find_all('tr')[1:]:
        cells = row.find_all('td')
        if len(cells) == len(headers):
            record = [cell.text.strip() for cell in cells]
            record_dict = dict(zip(headers, record))
            record_dict['trip_id'] = trip_id
            all_records.append(record_dict)

# Step 5: Convert to DataFrame
stops_df = pd.DataFrame(all_records)

# Step 6: Convert and clean column data types
stops_df['arrive_time'] = pd.to_numeric(stops_df['arrive_time'])
stops_df['tstamp'] = stops_df['arrive_time'].apply(lambda x: initial_date + timedelta(seconds=x))

stops_df['vehicle_number'] = stops_df['vehicle_number'].astype(str)
stops_df['trip_id'] = stops_df['trip_id'].astype(str)
stops_df['location_id'] = stops_df['location_id'].astype(str)
stops_df['ons'] = pd.to_numeric(stops_df['ons'], errors='coerce').fillna(0).astype(int)
stops_df['offs'] = pd.to_numeric(stops_df['offs'], errors='coerce').fillna(0).astype(int)

# Step 7: Reorder and select columns as required
stops_df = stops_df[['trip_id', 'vehicle_number', 'tstamp', 'location_id', 'ons', 'offs']]

# Step 8: Output assignment results
print(" DataFrame 'stops_df' created successfully.")
print(f"\nNumber of stop events: {len(stops_df)}")

num_vehicles = stops_df['vehicle_number'].nunique()
print(f"Number of Vehicles: {num_vehicles}")

stop_locations = stops_df['location_id'].nunique()
print(f"Number of Stop Locations: {stop_locations}")

min_tstamp = stops_df['tstamp'].min()
print(f"Minimum Timestamp: {min_tstamp}")

max_tstamp = stops_df['tstamp'].max()
print(f"Maximum Timestamp: {max_tstamp}")

boarding_stopevents = (stops_df['ons'] >= 1).sum()
print(f"Number of stop events with at least one passenger boarding: {boarding_stopevents}")

percentage_boarding_stopevents = (boarding_stopevents / len(stops_df)) * 100
print(f"Percentage of stop events with at least one passenger boarding: {percentage_boarding_stopevents:.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 DataFrame 'stops_df' created successfully.

Number of stop events: 93912
Number of Vehicles: 158
Number of Stop Locations: 4354
Minimum Timestamp: 2020-01-01 04:02:29
Maximum Timestamp: 2020-01-02 02:37:41
Number of stop events with at least one passenger boarding: 19858
Percentage of stop events with at least one passenger boarding: 21.15


In [8]:
print("\n  For location 6913:")

location_id = '6913'
location_df = stops_df[stops_df['location_id'] == location_id]

total_stops_at_location = location_df.shape[0]
print(f" Number of stops made at location {location_id}: {total_stops_at_location}")


unique_buses_at_location = location_df['vehicle_number'].nunique()
print(f" Number of different buses that stopped: {unique_buses_at_location}")


boarding_events_at_location = (location_df['ons'] >= 1).sum()
percentage_boarding_at_location = (boarding_events_at_location / total_stops_at_location) * 100
print(f" Percentage of stops with at least one boarding: {percentage_boarding_at_location:.2f}%")

print("\n  For vehicle 4062:")

vehicle_number = '4062'
vehicle_df = stops_df[stops_df['vehicle_number'] == vehicle_number]


total_stops_by_vehicle = vehicle_df.shape[0]
print(f" Number of stops made by vehicle {vehicle_number}: {total_stops_by_vehicle}")


total_boarded_by_vehicle = vehicle_df['ons'].sum()
print(f" Total passengers boarded: {total_boarded_by_vehicle}")


total_deboarded_by_vehicle = vehicle_df['offs'].sum()
print(f" Total passengers deboarded: {total_deboarded_by_vehicle}")

boarding_events_vehicle = (vehicle_df['ons'] >= 1).sum()
percentage_boarding_vehicle = (boarding_events_vehicle / total_stops_by_vehicle) * 100
print(f" Percentage of stops with at least one boarding: {percentage_boarding_vehicle:.2f}%")



  For location 6913:
 Number of stops made at location 6913: 15
 Number of different buses that stopped: 5
 Percentage of stops with at least one boarding: 13.33%

  For vehicle 4062:
 Number of stops made by vehicle 4062: 68
 Total passengers boarded: 26
 Total passengers deboarded: 26
 Percentage of stops with at least one boarding: 16.18%


In [9]:
! pip install scipy

In [12]:
from scipy.stats import binomtest

print("\n  Detect Vehicles with Biased Boarding Data")

# Step 4A: Calculate system-wide boarding rate (from Step 2E)
total_events = len(stops_df)
total_boarding_events = (stops_df['ons'] >= 1).sum()
system_boarding_rate = total_boarding_events / total_events
print(f"\nPercentage of stop events with boardings: {system_boarding_rate}")

# Step 4A: Analyze each vehicle
vehicle_stats = []

for vehicle_id, group in stops_df.groupby('vehicle_number'):
    n = len(group)  # total stops for this vehicle
    k = (group['ons'] >= 1).sum()  # stops with at least 1 boarding
    if n == 0:
        continue
    p_obs = k / n
    test_result = binomtest(k, n, system_boarding_rate, alternative='two-sided')

    vehicle_stats.append({
        'vehicle_number': vehicle_id,
        'total_stops': n,
        'boarding_stops': k,
        'boarding_rate': round(p_obs, 6),
        'p_value': test_result.pvalue
    })

# Step 4B: Create DataFrame and filter by p < 0.05
bias_df = pd.DataFrame(vehicle_stats)
biased_vehicles = bias_df[bias_df['p_value'] < 0.05].sort_values(by='p_value')

# Step 4B: Display formatted table
print("\nVehicles with biased boarding data p < 0.05:\n")
print(biased_vehicles[['vehicle_number', 'total_stops', 'boarding_stops', 'boarding_rate', 'p_value']].to_string(index=False))


print("\n Possible reasons for biased 'ons' (boarding) data")
print("1. Sensor malfunction: Faulty Automatic Passenger Counters may under- or over-record boardings.")
print("2. Route-specific behavior: Buses may be assigned to low-demand or express routes.")
print("3. Operational reasons: Vehicle may be used for training, maintenance, or non-revenue service.")



  Detect Vehicles with Biased Boarding Data

Percentage of stop events with boardings: 0.21145327540676379

Vehicles with biased boarding data p < 0.05:

vehicle_number  total_stops  boarding_stops  boarding_rate  p_value
          3915          662             115       0.173716 0.017249
          3530          576             100       0.173611 0.028077
          3963          405              68       0.167901 0.033011
          3733          709             128       0.180536 0.043074
          3634          409              70       0.171149 0.045715

 Possible reasons for biased 'ons' (boarding) data
1. Sensor malfunction: Faulty Automatic Passenger Counters may under- or over-record boardings.
2. Route-specific behavior: Buses may be assigned to low-demand or express routes.
3. Operational reasons: Vehicle may be used for training, maintenance, or non-revenue service.


In [15]:
from google.colab import drive
import pandas as pd
from scipy.stats import ttest_ind

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Load the Excel file (update path if needed)
relpos_path = '/content/drive/MyDrive/trimet_relpos_2022-12-07.csv'  # Ensure correct extension
df = pd.read_csv(relpos_path)

# Step 3: Normalize column names
df.columns = [col.lower().strip() for col in df.columns]
print("\n Columns in RELPOS dataset:", df.columns.tolist())

# Adjust column names if needed
vehicle_col = 'vehicle_number'  # update if your column is named differently
relpos_col = 'relpos'

# Step 4: Drop missing values
df = df.dropna(subset=[vehicle_col, relpos_col])

# Step 5: Get system-wide RELPOS values
global_relpos = df[relpos_col].astype(float).values

# Step 6: Analyze each vehicle
biased_vehicles = []

for vehicle_id, group in df.groupby(vehicle_col):
    vehicle_relpos = group[relpos_col].astype(float).values
    if len(vehicle_relpos) < 2:
        continue  # skip if not enough data
    t_stat, p_val = ttest_ind(vehicle_relpos, global_relpos, equal_var=False)

    if p_val < 0.005:
        biased_vehicles.append({
            'vehicle_id': vehicle_id,
            'sample_size': len(vehicle_relpos),
            'mean_relpos': round(vehicle_relpos.mean(), 6),
            'p_value': round(p_val, 6)
        })

# Step 7: Display results
biased_df = pd.DataFrame(biased_vehicles).sort_values(by='p_value')

print("\n Vehicles with biased RELPOS data (p < 0.005):")
if not biased_df.empty:
    print(biased_df.to_string(index=False))
else:
    print("No vehicles found with significant RELPOS bias.")

# Step 8: Print causes
print("\n Possible causes of biased RELPOS data:")
print("1. GPS signal obstruction: tall buildings, tunnels, or overpasses can interfere with GPS accuracy")
print("2. Map matching or routing errors: discrepancies between actual road geometry and the route model can create consistent offsets.")
print("3. Sensor misalignment or vehicle-specific hardware calibration issues, causing GPS antennas to report offset positions relative to the centerline.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

 Columns in RELPOS dataset: ['timestamp', 'vehicle_number', 'relpos']

 Vehicles with biased RELPOS data (p < 0.005):
 vehicle_id  sample_size  mean_relpos  p_value
       3638        10968     6.033647 0.000000
       3804        12491     5.977343 0.000000
       4305         5770    -6.171029 0.000000
       4024        12119    -0.180383 0.004008

 Possible causes of biased RELPOS data:
1. GPS signal obstruction: tall buildings, tunnels, or overpasses can interfere with GPS accuracy
2. Map matching or routing errors: discrepancies between actual road geometry and the route model can create consistent offsets.
3. Sensor misalignment or vehicle-specific hardware calibration issues, causing GPS antennas to report offset positions relative to the centerline.


In [16]:
from scipy.stats import chi2_contingency
import pandas as pd

print("\n Step: Detect Vehicles with Biased Boarding vs Deboarding (Chi-Square Test)")

# Step 1: System-wide totals
total_ons = stops_df['ons'].sum()
total_offs = stops_df['offs'].sum()
print(f"System Total ONs: {total_ons}, OFFs: {total_offs}")

# Step 2: Per-vehicle analysis
chi_results = []

for vehicle_id, group in stops_df.groupby('vehicle_number'):
    vehicle_ons = group['ons'].sum()
    vehicle_offs = group['offs'].sum()

    # Construct 2x2 contingency table:
    # [[vehicle_ons, vehicle_offs],
    #  [other_ons, other_offs]]
    other_ons = total_ons - vehicle_ons
    other_offs = total_offs - vehicle_offs

    contingency_table = [[vehicle_ons, vehicle_offs],
                         [other_ons, other_offs]]

    chi2, p, dof, expected = chi2_contingency(contingency_table)

    if p < 0.05:
        chi_results.append({
            'vehicle_id': vehicle_id,
            'vehicle_ons': vehicle_ons,
            'vehicle_offs': vehicle_offs,
            'p_value': round(p, 6)
        })

# Step 3: Show results
biased_chi_df = pd.DataFrame(chi_results).sort_values(by='p_value')

print("\n Vehicles with biased ons/offs proportions (p < 0.05):")
if not biased_chi_df.empty:
    print(biased_chi_df.to_string(index=False))
else:
    print("No vehicles with significant boarding/deboarding bias found.")

# Step 4: Print possible causes
print("\n Possible causes of biased ons/offs proportions:")
print("1. Riders may board a vehicle but transfer or exit on a different route/bus, skewing boarding vs deboarding counts.")
print("2. Some doors (like rear exits) may lack APC sensors, undercounting deboarding events.")
print("3. Sensor malfunction or calibration issues may over-record boardings or under-record exits.")



 Step: Detect Vehicles with Biased Boarding vs Deboarding (Chi-Square Test)
System Total ONs: 36373, OFFs: 36992

 Vehicles with biased ons/offs proportions (p < 0.05):
vehicle_id  vehicle_ons  vehicle_offs  p_value
      3576          379           322 0.018783
      3056          517           457 0.030134

 Possible causes of biased ons/offs proportions:
1. Riders may board a vehicle but transfer or exit on a different route/bus, skewing boarding vs deboarding counts.
2. Some doors (like rear exits) may lack APC sensors, undercounting deboarding events.
3. Sensor malfunction or calibration issues may over-record boardings or under-record exits.
